In [4]:
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer,CountVectorizer
from sklearn.metrics import confusion_matrix , roc_curve , auc
from nltk.stem.porter import PorterStemmer

In [5]:
con = sqlite3.connect('/home/arnab/Desktop/DATASETS/AmazonFineFoodReviews/database.sqlite')

Filtering only positive and negative reviews:

In [6]:
filtered_data = pd.read_sql_query("""SELECT*FROM Reviews WHERE Score!=3""",con)

In [7]:
def partition(x):
    if x<3:
        return 'negative'
    return 'positive'

In [8]:
actualScore = filtered_data['Score']
posNeg = actualScore.map(partition)
filtered_data['Score'] = posNeg

In [9]:
print(filtered_data.shape)
filtered_data.head()

(525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


***Cleaning the data:***

Removing Duplicates:

In [10]:
sorted_data = filtered_data.sort_values('ProductId',axis=0,ascending=True)

In [11]:
final= sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},keep="first",inplace=False)
final.shape

(364173, 10)

HelpfulnessDenominator>=HelpfulnessNumerator:

In [12]:
final=final[final.HelpfulnessDenominator>=final.HelpfulnessNumerator]

In [13]:
print(final.shape)
final['Score'].value_counts()

(364171, 10)


positive    307061
negative     57110
Name: Score, dtype: int64

***BAG OF WORDS:***

In [14]:
count_vect = CountVectorizer()
final_counts = count_vect.fit_transform(final['Text'].values)

In [15]:
print(type(final_counts))
final_counts.get_shape()

<class 'scipy.sparse._csr.csr_matrix'>


(364171, 115281)

***Text Preprocessing:***

In [16]:
import re
import nltk
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop = set(ENGLISH_STOP_WORDS)
sno = nltk.stem.SnowballStemmer('english')

def cleanhtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext=re.sub(cleanr,' ',sentence)
    return cleantext
def cleanpunc(sentence):
    cleaned=re.sub(r'[?|!|\'|"|#]',r' ',sentence)
    cleaned=re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return cleaned

print(stop)

{'amoungst', 'they', 'are', 'is', 'done', 'call', 'less', 'where', 'much', 'everything', 'not', 'get', 'hundred', 'to', 'therein', 'been', 'whereby', 'keep', 'ten', 'by', 'within', 'further', 'almost', 'hence', 'why', 'due', 'nevertheless', 'whether', 'fifteen', 'whereupon', 'thus', 'anyhow', 'your', 'there', 'front', 'back', 'thin', 'bottom', 'towards', 'should', 'seeming', 'anyway', 'whom', 'upon', 'former', 'afterwards', 'sincere', 'below', 'off', 'all', 'whatever', 'itself', 'seems', 'beside', 'else', 'mill', 'for', 'please', 'five', 'its', 'each', 'out', 'being', 'might', 'those', 'him', 'next', 'nothing', 'name', 'onto', 'themselves', 'be', 'full', 'meanwhile', 'together', 'do', 'thick', 'their', 'wherever', 'his', 'until', 'hers', 'always', 'mostly', 'it', 'cry', 'rather', 'third', 'something', 'so', 'once', 'everywhere', 'found', 'up', 'fire', 'take', 'have', 'part', 'fifty', 'three', 'two', 'cannot', 'he', 'was', 'still', 'at', 'along', 'throughout', 'also', 'bill', 'becoming'

In [17]:
i=0
strl=' '
final_string=[]
all_pos_words=[]
all_neg_words=[]
s=''
for sent in final['Text'].values:
    filtered_sent=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha())&(len(cleaned_words)>2)):
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sent.append(s)
                    if(final['Score'].values)[i] == 'positive':
                        all_pos_words.append(s)
                    if(final['Score'].values)[i] == 'negative': 
                        all_neg_words.append(s)
                else:
                    continue
            else:
                continue
    strl =b" ".join(filtered_sent)
    final_string.append(strl)
    i+=1                                    

In [18]:
final['CleanedText']=final_string
final['CleanedText'].head()

138706    b'witti littl book make son laugh loud recit c...
138688    b'grew read sendak book watch realli rosi movi...
138689    b'fun way children learn month year learn poem...
138690    b'great littl book read nice rhythm good repet...
138691    b'book poetri month year goe month cute littl ...
Name: CleanedText, dtype: object

In [19]:
conn = sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory=str
final.to_sql('Reviews',conn,schema=None,if_exists='replace')

364171

***BI-GRAMS and n-GRAMS:***

In [20]:
freq_dist_pos=nltk.FreqDist(all_pos_words)
freq_dist_neg=nltk.FreqDist(all_neg_words)
print(freq_dist_pos.most_common(10))
print(freq_dist_neg.most_common(10))

[(b'like', 139432), (b'tast', 129068), (b'good', 112789), (b'flavor', 109637), (b'love', 107396), (b'great', 103924), (b'use', 103893), (b'product', 91054), (b'just', 88378), (b'tri', 86803)]
[(b'tast', 34592), (b'like', 32331), (b'product', 28227), (b'flavor', 19580), (b'just', 18763), (b'tri', 17756), (b'use', 15303), (b'good', 15044), (b'coffe', 14720), (b'buy', 13752)]


In [21]:
count_vect=CountVectorizer(ngram_range=(1,2))
final_bigrams_counts=count_vect.fit_transform(final['Text'].values)

In [22]:
final_bigrams_counts.get_shape()

(364171, 2910192)

***TF-IDF:***

In [23]:
tf_idf_vect=TfidfVectorizer(ngram_range=(1,2))
final_tf_idf=tf_idf_vect.fit_transform(final['Text'].values)

In [24]:
final_tf_idf.get_shape()

(364171, 2910192)

In [25]:
features = tf_idf_vect.get_feature_names_out()
len(features)

2910192

In [26]:
features[100000:100010]

array(['ales until', 'ales ve', 'ales would', 'ales you', 'alessandra',
       'alessandra ambrosia', 'alessi', 'alessi added', 'alessi also',
       'alessi and'], dtype=object)

In [27]:
print((final_tf_idf[3,:]).toarray()[0])

[0. 0. 0. ... 0. 0. 0.]


"***IMP"Getting Top Features:

In [28]:
def top_tf_idf_feats(rows,features,n=25):
    topn_ids=np.argsort(rows)[::-1][:n]
    top_feats=[(features[i],rows[i])for i in topn_ids]
    df=pd.DataFrame(top_feats)
    df.columns=['feature','tfidf']
    return df
top_tfidf=top_tf_idf_feats(final_tf_idf[1,:].toarray()[0],features,25)

In [29]:
top_tfidf.head()

,feature,tfidf
0,sendak books,0.173437
1,rosie movie,0.173437
2,paperbacks seem,0.173437
3,cover version,0.173437
4,these sendak,0.173437
